# <center>Trabajo especial N° 2: Cuadrados mínimos</center>

Dadas $m$ funciones modelo, $\phi_1(x), \dots, \phi_m(x), \ x \in \mathbb{R}$ y un conjunto de pares de números reales $(x_i, y_i)$, con $i=1, \dots, n$, queremos encontrar una combinación lineal de las funciones modelo de manera tal de que en cada $x_i$ valga aproximadamente $y_i$. Es decir, buscamos $v_1, \dots, v_n$ tales que

$$
\begin{bmatrix}
           y_1 \\
           \vdots \\
           y_n
\end{bmatrix} \approx 
\begin{bmatrix}
           \sum_j v_j \phi_j(x_1)\\
           \vdots \\
           \sum_j v_j \phi_j(x_n)
\end{bmatrix}.
$$

Observemos que el vector del lado derecho de la igualdad lo podemos escribir de la forma $Av$ donde $v=(v_1, \dots, v_n)$ y

$$
A = \begin{bmatrix}
           \phi_1(x_1), \dots, \phi_m(x_1)\\
           \phi_1(x_2), \dots, \phi_m(x_2)\\
           \vdots \\
           \phi_1(x_n), \dots, \phi_m(x_n)\\ 
\end{bmatrix}
$$

Así, para resolver nuestro problema queremos encontrar el $v$ que minimiza la diferencia entre $Av$ e $y = (y_1, \dots, y_n)$, o equivalentemente, que minimice $|| Av - y ||^2$. Es decir queremos la mejor aproximación posible a $y$ dentro del subespacio $S = \{ Av: v \in \mathbb{R}^m \}$. Se trata de la proyección ortogonal de $y$ sobre $S$ que es única siempre y cuando que se trate de un subespacio de un espacio vectorial de dimensión finita con producto interno.

Notemos que si queremos el $v$ que minimiza $||Av-y||^2$, tenemos


$$
\begin{split}
||Av-y||^2 & = \langle Av-y, Av-y \rangle \\
& = \langle Av-y, Av \rangle - \langle Av-y, y \rangle \\
& = \langle Av, Av \rangle - 2\langle Av, y \rangle + ||y||^2 \\
& = \langle A^t Av, v \rangle - 2\langle v,A^ty \rangle + ||y||^2.
\end{split}
$$

Como $y$ es en este caso un valor conocido (constante), nos bastará con hallar el $v$ que minimice $ \frac{1}{2}\langle A^tAv,v \rangle - \langle A^ty,v \rangle.$

Para hallar las condiciones de primer orden diferenciamos esta expresión e igualamos a cero:
$$
\begin{split}
& \bigg(\frac{1}{2}\langle A^tAv,v \rangle - \langle A^ty,v \rangle\bigg)' = 0 \\
& \iff \frac{1}{2}(A^tAv + A^Av) - A^ty = 0 \\
& \iff A^tAv = A^ty.
\end{split}
$$

En el caso de que $rg(A)=m$ tenemos que $A$ es una matriz inversible, por lo cual $A^tA$ también lo es (el determinante es distributivo con el producto matricial e invariante bajo la trasposición). Así, $x = (A^tA)^{-1}A^ty$ resulta la única solución del sistema $A^tAx = A^ty$. En el caso contrario, es decir cuando $rg(A)<m$, podemos generar fácilmente infinitas soluciones con combinaciones lineales de algún vector $u \in Ker(A^tA)$.

Si $\hat{x} \in S$ es la proyección ortogonal de $y$ sobre $S$, entonces $y-\hat{x}$ es ortogonal a todos los elementos de $S$, entonces el teorema de pitágores nos dice que para todo $x \in S, ||\hat{x}||^2 + ||y-\hat{x}||^2 = ||y-x||^2$ lo que implica que $||\hat{x} - y|| \leq ||x-y|| \ \forall x \in S$. Cualquier otra otra proyección ortogonal de $y$ sobre $S$ no cumpliría con lo anterior, por lo que no puede haber dos.

In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [15]:
def noisy_arc(
    nodes=1000,
    r0=1,
    x0=0,
    y0=0,
    theta0=0,
    theta1=np.pi/6,
    delta_r=0.05
):
    # cloud
    theta = np.random.uniform(low=theta0, high=theta1, size=nodes)
    radius = np.random.normal(loc=r0, scale=delta_r, size=nodes)
    x, y = radius * np.cos(theta) + x0, radius * np.sin(theta) + y0
    
    # real circle
    full_theta = np.linspace(theta0, theta1, 10000)
    x_true, y_true = r0 * np.cos(full_theta) + x0, r0 * np.sin(full_theta) + y0
    
    # fit by OLS
    matrix = np.array([x, y, np.ones(nodes)]).T
    b =  x**2 + y**2
    res = np.linalg.lstsq(matrix, b, rcond=None)
    x0_hat, y0_hat, r0_hat = res[0][0], res[0][1], res[0][2]
    print(
        f"x0: {x0_hat.round(3)}, y0: {y0_hat.round(3)}, r0: {r0_hat.round(3)}."
    )
    x_hat, y_hat = r0_hat * np.cos(full_theta) + y0_hat, r0_hat * np.sin(full_theta) + y0_hat
    
    # plot
    fig = make_subplots(
        rows=2, cols=2, subplot_titles=('Data', 'r(theta)', 'Radius', 'Theta')
    )
    fig.add_trace(go.Scatter(x=x, y=y, mode='markers'), row=1, col=1)
    fig.add_trace(go.Scatter(x=x_true, y=y_true, mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=x_hat, y=y_hat, mode='lines'), row=1, col=1)
    fig.add_trace(go.Scatter(x=theta, y=radius, mode='markers'), row=1, col=2)
    fig.add_trace(go.Histogram(x=radius), row=2, col=1)
    fig.add_trace(go.Histogram(x=theta), row=2, col=2)
    fig.update_layout(autosize=False, width=966, height=966, showlegend=False)
    
    fig.show()
    
    return res[0]

In [16]:
coeffs = noisy_arc()

x0: 1.879, y0: 0.501, r0: -0.92.
